In [ ]:
%env MUJOCO_GL=egl

import cv2
import time
import mujoco
import numpy as np
from dlabsim.envs.mmk2_base import MMK2Base, MMK2Cfg

In [ ]:
# 配置文件
cfg = MMK2Cfg()
cfg.render_set["height"] = 480 # 渲染窗口高度
cfg.render_set["width"]  = 640 # 渲染窗口宽度
cfg.mjcf_file_path = "mjcf/exhibition.xml" # mjcf模型文件路径 models路径下
cfg.use_gaussian_renderer = False   # 不使用高斯渲染器
cfg.headless = True                 # 不显示窗口
cfg.obs_camera_id = [-1]            # 相机id
cfg.init_key = "front_table"        # 初始化位姿

# 创建环境
env = MMK2Base(cfg)
# 设置渲染标志 mjVIS_CONTACTFORCE 显示接触力
env.options.flags[mujoco.mjtVisFlag.mjVIS_CONTACTFORCE] = True

# 设置相机视角
env.free_camera.lookat[:] = env.getObjPose("head_cam")[0]
env.free_camera.lookat[2] -= 0.5
env.free_camera.distance = 2

# 重置环境
obs = env.reset()
# 显示观测
print("obs.keys() = ", *obs.keys())
for k, v in obs.items():
    if "j" in k:
        print(k, v)
img = obs["img"][-1]
cv2.imshow("img", cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
class CollisionDetector:
    # def __init__(self, mjcf_file) -> None:
        # self.mj_model = mujoco.MjModel.from_xml_path(mjcf_file)
    def __init__(self, mj_model) -> None:
        self.mj_model = mj_model
        self.mj_data_col = mujoco.MjData(self.mj_model)

    def check_collision(self, qpos):
        """ 
        Check if the robot is in collision with the environment.
        Args:
            qpos (np.ndarray): The joint positions of the robot.
        Returns:
            bool: True if the robot is in collision with the environment.
        """
        self.mj_data_col.qpos[:] = qpos[:]
        mujoco.mj_forward(self.mj_model, self.mj_data_col)
        # 0是地面id 计算除了地面以外的碰撞
        return bool(np.where(self.mj_data_col.contact.geom[:,0] != 0)[0].shape[0])

# cls_det = CollisionDetector(env.mjcf_file)
cls_det = CollisionDetector(env.mj_model)
cls_ret = cls_det.check_collision(obs["jq"])

In [ ]:
# 移动方块 检测碰撞
env.mj_model.body("red_box").pos[0] = -0.7
for i in range(100):
    env.mj_model.body("red_box").pos[0] -= 0.01
    mujoco.mj_forward(env.mj_model, env.mj_data)
    env.render()
    obs = env.getObservation()
    cls_ret = cls_det.check_collision(obs["jq"])
    img_show = cv2.cvtColor(obs["img"][-1], cv2.COLOR_RGB2BGR)
    if cls_ret:
        cv2.putText(img_show, "Collision", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("img", img_show)
    key = cv2.waitKey(1000//30)
    if key == 27:
        break
cv2.destroyAllWindows()


In [ ]:
# 移动机器人 检测碰撞
env.reset()

mj_base = env.mj_data.qpos[:7]
mj_wheel_joint = env.mj_data.qpos[7:9]
mj_slide_joint = env.mj_data.qpos[9:10]
mj_head_joint = env.mj_data.qpos[10:12]
mj_left_arm_joint = env.mj_data.qpos[12:18]
mj_left_gripper_joint = env.mj_data.qpos[18:20]
mj_right_arm_joint = env.mj_data.qpos[20:26]
mj_right_gripper_joint = env.mj_data.qpos[26:28]

steps = 100
env.mj_model.body("red_box").pos[0] = -0.7
for i in range(steps):
    mj_left_arm_joint[1] += 0.8/steps
    mujoco.mj_forward(env.mj_model, env.mj_data)
    env.render()
    obs = env.getObservation()
    img_show = cv2.cvtColor(obs["img"][-1], cv2.COLOR_RGB2BGR)
    cls_ret = cls_det.check_collision(env.mj_data.qpos)
    if cls_ret:
        cv2.putText(img_show, "Collision", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow("img", img_show)
    key = cv2.waitKey(1000//30)
    if key == 27:
        break
cv2.destroyAllWindows()

